<a href="https://colab.research.google.com/github/madixson/DogsvsCats_4380Project/blob/main/VGG16Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Pretrained VGG16 Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Set up data generators for training and validation
train_dir = '/content/drive/MyDrive/DATA_4380/dataset_dogs_vs_cats/train/'
test_dir = '/content/drive/MyDrive/DATA_4380/dataset_dogs_vs_cats/test/'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

# Load pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new top layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine base model with new top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history_vgg16 = model.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size,
          epochs=10, validation_data=test_generator, validation_steps=test_generator.n//test_generator.batch_size)

Found 18796 images belonging to 2 classes.
Found 6344 images belonging to 2 classes.
58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
587/587 [==============================] - 17103s 29s/step - loss: 0.3885 - accuracy: 0.8514 - val_loss: 0.2070 - val_accuracy: 0.9165
Epoch 2/10
587/587 [==============================] - 16066s 27s/step - loss: 0.2603 - accuracy: 0.8872 - val_loss: 0.1720 - val_accuracy: 0.9287
Epoch 3/10
587/587 [==============================] - 15963s 27s/step - loss: 0.2375 - accuracy: 0.8981 - val_loss: 0.1843 - val_accuracy: 0.9238
Epoch 4/10
587/587 [==============================] - 15924s 27s/step - loss: 0.2322 - accuracy: 0.9001 - val_loss: 0.1654 - val_accuracy: 0.9299
Epoch 5/10
120/587 [=====>........................] - ETA: 2:37:28 - loss: 0.2150 - accuracy: 0.9102

## Save results
Use pickle package to save training results for analysis result

In [ ]:
import pickle

with open('/content/drive/MyDrive/DATA_4380/history_vgg16.pkl', 'wb') as f:
    pickle.dump(history_vgg16.history, f)